In [5]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# changed animal_shelter and AnimalShelter to match CRUD Python file.
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Added username and password.

username = "aacuser2"
password = "password"
HOST = 'nv-desktop-services.apporto.com'
PORT = 31269
DB = 'AAC'
COL = 'animals'

# added CRUD Python file name
shelter = AnimalShelter(username, password)


# Class read method which filters for Dogs.
df = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog'}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Client's logo imported - Grazioso Salvare.’
image_filename = 'GraziosoSalvareLogo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# HTML for headline display
app.layout = html.Div([
    html.Div(className='row',
        style={'display' : 'flex', 'margin': '10px'},
            children=[
                # HTML for client logo
                html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={'height' : '8%', 'width': '8%'}),
                # HTML for headline title
                html.Center(html.B(html.H1('Grazioso Salvare Dashboard - Melissa Cianfarao')))
            ]),          
                
    html.Hr(),
    html.Div(
        
# Radio buttons in a row for filter options. 
    dcc.RadioItems(
        id='filter-type',
        # Establishing radio button types with labels.
        options=[
            {'label': '  Water Rescue   ', 'value': 'Water'},
            {'label': '  Mountain and Wilderness Rescue   ', 'value': 'Mountain'},
            {'label': '  Disaster and Individual Rescue   ', 'value': 'Disaster'},
            {'label': '  Reset', 'value': 'Reset'}   
        ],
        value='Reset'
    )
    ),
    html.Hr(),
    # Interactive table features.
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable="single",
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type is None:
        return
    
    df = None
    
    # Filter settings for Water rescue dogs with breeds, sex upon outcome, and age range.
    if filter_type == 'Water':
        df = pd.DataFrame.from_records(shelter.read(
            { "$and": [
                {"$or": [
                    {'breed': {"$regex": 'Labrador Retriever Mix'}},
                    {'breed': {"$regex": 'Chesapeake'}},
                    {'breed': {"$regex": 'Newfoundland'}}]},
                {'sex_upon_outcome': 'Intact Female'},
                {"$and": [
                    {'age_upon_outcome_in_weeks':{"$gt": 26}},
                    {'age_upon_outcome_in_weeks':{"$lt": 156}}
                ]}    
            ]}))
    # Filter settings for Mountain rescue dogs with breeds, sex upon outcome, and age range.
    elif filter_type == 'Mountain':
        df = pd.DataFrame.from_records(shelter.read(
            { "$and": [
                {"$or": [
                    {'breed': {"$regex": 'German Shepherd'}},
                    {'breed': {"$regex": 'Alaskan Malamute'}},
                    {'breed': {"$regex": 'Old English Sheepdog'}},
                    {'breed': {"$regex": 'Siberian Husky'}},
                    {'breed': {"$regex": 'Rottweiler'}}]},
                {'sex_upon_outcome': 'Intact Male'},
                {"$and": [
                    {'age_upon_outcome_in_weeks':{"$gt": 26}},
                    {'age_upon_outcome_in_weeks':{"$lt": 156}}
                ]}          
            ]}))
    # Filter settings for Disaster rescue dogs with breeds, sex upon outcome, and age range.
    elif filter_type == 'Disaster':
        df = pd.DataFrame.from_records(shelter.read(
            { "$and": [
                {"$or": [
                    {'breed': {"$regex": 'German Shepard'}},
                    {'breed': {"$regex": 'Alaskan Malamute'}},
                    {'breed': {"$regex": 'Old English Sheepdog'}},
                    {'breed': {"$regex": 'Siberian Husky'}},
                    {'breed': {"$regex": 'Rottweiler'}}]},
                {'sex_upon_outcome': 'Intact Male'},
                {"$and": [
                    {'age_upon_outcome_in_weeks':{"$gt": 26}},
                    {'age_upon_outcome_in_weeks':{"$lt": 156}}
                ]}               
            ]}))
    # Filters for reset to show all dog types.            
    else:
        df = pd.DataFrame.from_records(shelter.read({'animal_type':'Dog'}))
                
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')
            
    
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # Code for displaying pie chart.
    if viewData is None:
        return
    
    df = pd.DataFrame.from_records(viewData)
    return [
        dcc.Graph(
            # Titles for pie chart
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '900px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:13843/
